![tracker](https://us-central1-vertex-ai-mlops-369716.cloudfunctions.net/pixel-tracking?path=statmike%2Fvertex-ai-mlops%2FDev&file=AutoML+to+Cloud+Run.ipynb)
<!--- header table --->
<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/statmike/vertex-ai-mlops/blob/main/Dev/AutoML%20to%20Cloud%20Run.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo">
      <br>Run in<br>Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https%3A%2F%2Fraw.githubusercontent.com%2Fstatmike%2Fvertex-ai-mlops%2Fmain%2FDev%2FAutoML%2520to%2520Cloud%2520Run.ipynb">
      <img width="32px" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" alt="Google Cloud Colab Enterprise logo">
      <br>Run in<br>Colab Enterprise
    </a>
  </td>      
  <td style="text-align: center">
    <a href="https://github.com/statmike/vertex-ai-mlops/blob/main/Dev/AutoML%20to%20Cloud%20Run.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      <br>View on<br>GitHub
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/statmike/vertex-ai-mlops/main/Dev/AutoML%20to%20Cloud%20Run.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      <br>Open in<br>Vertex AI Workbench
    </a>
  </td>
</table>

Export Tabular Model:
- https://cloud.google.com/vertex-ai/docs/export/export-model-tabular
- https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/gapic/automl/showcase_automl_tabular_classification_export_cloud.ipynb

Note: the correct serving container is listed in the environment.json file in the export

Next step: Host on Cloud Run
Ideation:
- https://medium.com/google-cloud/portable-prediction-with-tensorflow-and-cloud-run-669c1c73ebd1
- https://blog.verta.ai/how-to-deploy-ml-models-with-google-cloud-run


Resource: Python Vertex API
https://googleapis.dev/python/aiplatform/latest/aiplatform_v1/types.html#google.cloud.aiplatform_v1.types.ExportModelRequest.OutputConfig


In [169]:
PROJECT_ID = 'statmike-demo3'
REGION = 'us-central1'
DATANAME = 'fraud'
NOTEBOOK = 'automl_export'
DOCKER = "us-docker.pkg.dev/vertex-ai/automl-tabular/prediction-server-v1"

In [108]:
PARENT = f"projects/{PROJECT_ID}/locations/{REGION}"
URI = f"gs://{PROJECT_ID}/{DATANAME}/{NOTEBOOK}/models"
DIR = f"temp/{NOTEBOOK}"

In [82]:
import google.cloud.aiplatform_v1 as vertex
from google.cloud import bigquery

In [109]:
!rm -rf {DIR}
!mkdir -p {DIR}

E0413 18:34:00.911505557    7868 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies
E0413 18:34:01.049796719    7868 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies


In [83]:
client_options = {"api_endpoint": f"{REGION}-aiplatform.googleapis.com"}

bigquery = bigquery.Client()

In [84]:
modelName = '02a_202215193614' # '04g_tournament_automl_28'  # '02a_202215193614'
request = vertex.ListModelsRequest(parent = PARENT, filter = f"display_name={modelName}")

In [85]:
client_models = vertex.ModelServiceClient(client_options = client_options)
models = client_models.list_models(request = request)

In [86]:
for model in models:
    model_id = model.name

In [87]:
model_id

'projects/715288179162/locations/us-central1/models/5979693987659776000'

In [88]:
gcs_destination = {"output_uri_prefix" : URI}
output_config = {
    "artifact_destination" : gcs_destination,
    "export_format_id" : "tf-saved-model"
}

In [90]:
response = client_models.export_model(name = model_id, output_config = output_config)

In [93]:
response.result(timeout = 100)

In [145]:
result = response.operation.metadata
result

type_url: "type.googleapis.com/google.cloud.aiplatform.v1.ExportModelOperationMetadata"
value: "\n\034\022\014\010\225\230\334\222\006\020\370\331\237\210\003\032\014\010\234\230\334\222\006\020\260\261\302\301\001\022u\022sgs://statmike-demo3/fraud/automl_export/models/model-5979693987659776000/tf-saved-model/2022-04-13T17:44:53.737907Z"

In [150]:
str(result.value).split('\\')[-1][4:-1]

'gs://statmike-demo3/fraud/automl_export/models/model-5979693987659776000/tf-saved-model/2022-04-13T17:44:53.737907Z'

In [142]:
response.operation.name

'projects/715288179162/locations/us-central1/models/5979693987659776000/operations/8238062902284648448'

In [151]:
URI

'gs://statmike-demo3/fraud/automl_export/models'

In [156]:
URI_MODEL = str(result.value).split('\\')[-1][4:-1]
URI_MODEL

'gs://statmike-demo3/fraud/automl_export/models/model-5979693987659776000/tf-saved-model/2022-04-13T17:44:53.737907Z'

In [157]:
!ls

'08a - Vertex AI - Pipelines - BQML 02+03.ipynb'  'forecasting master.ipynb'
'AutoML to Cloud Run.ipynb'			   models
'Custom Managed Notebook.ipynb'			   pyspark_ml_lr_dataproc.ipynb
'Notes - Just Notes.ipynb'			   sample_kmeans_data.txt
'aa - TFX on Vertex AI - Interactive.ipynb'	  'sparkml gmm test.ipynb'
 dataproc_gmm.ipynb				   temp
 dataproc_lr.ipynb


E0414 09:56:01.305501086    7868 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies


In [158]:
!rm -r models

E0414 09:56:04.558637516    7868 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies


In [159]:
!gsutil cp -r {URI_MODEL} .

E0414 09:56:56.495273682    7868 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies


Copying gs://statmike-demo3/fraud/automl_export/models/model-5979693987659776000/tf-saved-model/2022-04-13T17:44:53.737907Z/environment.json...
Copying gs://statmike-demo3/fraud/automl_export/models/model-5979693987659776000/tf-saved-model/2022-04-13T17:44:53.737907Z/feature_attributions.yaml...
Copying gs://statmike-demo3/fraud/automl_export/models/model-5979693987659776000/tf-saved-model/2022-04-13T17:44:53.737907Z/final_model_structure.pb...
Copying gs://statmike-demo3/fraud/automl_export/models/model-5979693987659776000/tf-saved-model/2022-04-13T17:44:53.737907Z/instance.yaml...
/ [4 files][  8.4 KiB/  8.4 KiB]                                                
==> NOTE: You are performing a sequence of gsutil operations that may
run significantly faster if you instead use gsutil -m cp ... Please
see the -m section under "gsutil help options" for further information
about when gsutil -m can be advantageous.

Copying gs://statmike-demo3/fraud/automl_export/models/model-5979693987659776

In [160]:
!ls

'08a - Vertex AI - Pipelines - BQML 02+03.ipynb'   dataproc_lr.ipynb
 2022-04-13T17:44:53.737907Z			  'forecasting master.ipynb'
'AutoML to Cloud Run.ipynb'			   pyspark_ml_lr_dataproc.ipynb
'Custom Managed Notebook.ipynb'			   sample_kmeans_data.txt
'Notes - Just Notes.ipynb'			  'sparkml gmm test.ipynb'
'aa - TFX on Vertex AI - Interactive.ipynb'	   temp
 dataproc_gmm.ipynb


E0414 09:57:04.189815472    7868 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies


In [163]:
ts = URI_MODEL.split("/")[-1]
ts

'2022-04-13T17:44:53.737907Z'

In [164]:
!mv $ts model

E0414 09:58:26.035975557    7868 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies


In [166]:
!ls

'08a - Vertex AI - Pipelines - BQML 02+03.ipynb'  'forecasting master.ipynb'
'AutoML to Cloud Run.ipynb'			   model
'Custom Managed Notebook.ipynb'			   pyspark_ml_lr_dataproc.ipynb
'Notes - Just Notes.ipynb'			   sample_kmeans_data.txt
'aa - TFX on Vertex AI - Interactive.ipynb'	  'sparkml gmm test.ipynb'
 dataproc_gmm.ipynb				   temp
 dataproc_lr.ipynb


E0414 09:58:36.200449198    7868 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies


In [167]:
!ls model

environment.json	   instance.yaml	   tables_server_metadata.pb
feature_attributions.yaml  predict		   transformations.pb
final_model_structure.pb   prediction_schema.yaml


E0414 09:58:47.064078712    7868 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies


In [170]:
!docker pull {DOCKER}

Using default tag: latest
latest: Pulling from vertex-ai/automl-tabular/prediction-server-v1
Digest: sha256:9d83d303f8d2414c070490d16144ef68e0d8b1385ea067dbe145804087b36e9b
Status: Image is up to date for us-docker.pkg.dev/vertex-ai/automl-tabular/prediction-server-v1:latest
us-docker.pkg.dev/vertex-ai/automl-tabular/prediction-server-v1:latest


E0414 10:03:06.750039686    7868 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies


---

In [227]:
env = open('model/environment.json')
env_data = json.load(env)
env_data['container_uri']

'us-docker.pkg.dev/vertex-ai/automl-tabular/prediction-server:20211210_1325_RC00'

In [228]:
# test environment.json image
!docker pull {env_data['container_uri']}

E0414 11:55:02.489696211    7868 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies


20211210_1325_RC00: Pulling from vertex-ai/automl-tabular/prediction-server
Digest: sha256:1cbce3e92fbef4043e8fcf22766abfef841ed5ba30878e7fb8fffc68936e54c7
Status: Image is up to date for us-docker.pkg.dev/vertex-ai/automl-tabular/prediction-server:20211210_1325_RC00
us-docker.pkg.dev/vertex-ai/automl-tabular/prediction-server:20211210_1325_RC00


In [209]:
# run old image
docker_container_id = ! docker run -d -v `pwd`/model:/models/default -p 8081:8080 -it {env_data['container_uri']}

E0414 11:29:31.169608053    7868 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies


---

In [171]:
docker_container_id = ! docker run -d -v `pwd`/model:/models/default -p 8081:8080 -it {DOCKER}

E0414 10:03:54.476292168    7868 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies


---

empty health check means good

In [211]:
!curl -X GET http://localhost:8081/health

E0414 11:29:47.113749205    7868 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies


In [212]:
!docker container ls --latest

CONTAINER ID   IMAGE                                                                             COMMAND                  CREATED          STATUS          PORTS                    NAMES
082865c58699   us-docker.pkg.dev/vertex-ai/automl-tabular/prediction-server:20211210_1325_RC00   "python3 -m google3.…"   18 seconds ago   Up 12 seconds   0.0.0.0:8081->8080/tcp   laughing_mahavira


E0414 11:29:49.355510840    7868 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies


In [213]:
pred = bigquery.query(query = f"SELECT * FROM {DATANAME}.{DATANAME}_prepped WHERE splits='TEST' LIMIT 10").to_dataframe()
newob = pred[pred.columns[~pred.columns.isin(['Class','transaction_id', 'splits'])]].to_dict(orient='records')[0]
newob['Time'] = str(newob['Time'])
newob = {'instances': [newob]}
newob

{'instances': [{'Time': '35271',
   'V1': 1.05515950969377,
   'V2': -0.0239651057105096,
   'V3': 1.57533166546303,
   'V4': 2.85974528092561,
   'V5': -0.997167571777128,
   'V6': 0.561406984887084,
   'V7': -0.808138197636819,
   'V8': 0.423217410023693,
   'V9': 0.41247329364736296,
   'V10': 0.596773097767931,
   'V11': 0.266905042003122,
   'V12': 0.287455425475823,
   'V13': -1.74122657483201,
   'V14': -0.217416020840106,
   'V15': -2.03782438994774,
   'V16': 0.237223007701601,
   'V17': -0.0874920123721036,
   'V18': 0.30262016786209,
   'V19': -0.34309649976518297,
   'V20': -0.27971192424531,
   'V21': 0.0685928085837315,
   'V22': 0.449717577749804,
   'V23': -0.0978070799273023,
   'V24': 0.5504948912296079,
   'V25': 0.509526617338925,
   'V26': 0.187886484081211,
   'V27': 0.0354154015648864,
   'V28': 0.0156355109900566,
   'Amount': 0.0}]}

In [214]:
import json
with open(f'{DIR}/request.json', 'w') as f:
    data = json.dumps(newob)
    f.write(data)

In [218]:
!curl -X POST --data @{DIR}/request.json http://localhost:8081/predict

{"predictions": [{"classes": ["0", "1"], "scores": [0.9177474975585938, 0.08225249499082565]}]}

E0414 11:50:06.189549124    7868 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies


In [219]:
docker_container_id

['082865c5869980fa753389c04b1a4caae610a9312d2715751cd0e2b691531848']

In [220]:
!docker container ls

CONTAINER ID   IMAGE                                                                             COMMAND                  CREATED          STATUS          PORTS                    NAMES
082865c58699   us-docker.pkg.dev/vertex-ai/automl-tabular/prediction-server:20211210_1325_RC00   "python3 -m google3.…"   21 minutes ago   Up 20 minutes   0.0.0.0:8081->8080/tcp   laughing_mahavira
3c99c43d5344   gcr.io/inverting-proxy/agent                                                      "/bin/sh -c '/opt/bi…"   2 weeks ago      Up 2 weeks                               proxy-agent


E0414 11:50:33.547059923    7868 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies


In [221]:
docker_container_id[0][0:12]

'082865c58699'

In [222]:
!docker kill {docker_container_id[0][0:12]}

E0414 11:50:43.650495772    7868 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies


082865c58699


In [223]:
!docker container ls

CONTAINER ID   IMAGE                          COMMAND                  CREATED       STATUS       PORTS     NAMES
3c99c43d5344   gcr.io/inverting-proxy/agent   "/bin/sh -c '/opt/bi…"   2 weeks ago   Up 2 weeks             proxy-agent


E0414 11:50:45.698976669    7868 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies
